In [1]:
def solve_cryptarithmetic(puzzle):
    words = puzzle[:-1].split('+')
    result_word = puzzle[-1]
    chars = set(''.join(words) + result_word)

    def is_valid_assignment(assignment):
        return sum(assignment[word[0]] for word in words) == assignment[result_word[0]]

    def is_consistent(char, digit, assignment):
        return char not in assignment or assignment[char] == digit

    def propagate_constraints(assignment):
        updated = True
        while updated:
            updated = False
            for word in words:
                if len(word) == 1:  # Si la palabra tiene solo un carácter, no se pueden propagar restricciones
                    continue
                if all(c in assignment for c in word[:-1]):
                    last_char = word[-1]
                    if last_char not in assignment:
                        possible_digits = [int(d) for d in range(10) if d not in assignment.values()]
                        consistent_digits = [d for d in possible_digits if is_consistent(last_char, d, assignment)]
                        if len(consistent_digits) == 1:
                            assignment[last_char] = consistent_digits[0]
                            updated = True

    def select_next_char(assignment):
        unassigned_chars = chars - assignment.keys()
        unassigned_chars = sorted(unassigned_chars, key=lambda char: calculate_degree(char), reverse=True)
        return unassigned_chars[0] if unassigned_chars else None

    def calculate_degree(char):
        return sum(1 for word in words if char in word)

    def backtrack_search(assignment):
        if len(assignment) == len(chars):
            if is_valid_assignment(assignment):
                return assignment
            else:
                return None

        char = select_next_char(assignment)
        if char is None:
            return None

        for digit in range(10):
            if is_consistent(char, digit, assignment):
                assignment[char] = digit
                propagate_constraints(assignment)  # Propagar restricciones después de cada asignación
                result = backtrack_search(assignment)
                if result:
                    return result
                assignment.pop(char)
        return None

    solution = backtrack_search({})
    return solution

# Example usage
if __name__ == "__main__":
    puzzle = "SEND+MORE=MONEY"
    solution = solve_cryptarithmetic(puzzle)
    if solution:
        print("Solución encontrada:")
        for char, digit in solution.items():
            print(f"{char}: {digit}")
    else:
        print("No se encontró solución.")

Solución encontrada:
N: 0
E: 0
D: 0
O: 0
R: 0
S: 0
=: 0
M: 0
Y: 0


In [10]:
equation = "SEND+MORE=MONEY"
solution = solve_cryptarithmetic(equation)

if solution:
  print("Solución encontrada:")
  for char, digit in solution.items():
    print(f"{char}: {digit}")
else:
  print("No se encontró solución.")


Solución encontrada:
Y: 0
N: 0
D: 0
E: 0
O: 0
R: 0
S: 0
=: 0
M: 0


In [18]:
import random 

In [ ]:
#### 

In [48]:
class Variable:
    def __init__(self, name):
        self.name = name
        self.domain = set(range(10))
        self._assigned = False

    def assigned(self):
        return self._assigned

    def unassign(self):
        self._assigned = False

    def assign(self, value):
        self.domain = {value}
        self._assigned = True
        

class Constraint:
    def __init__(self, variable1, position, value):
        self.variable1 = variable1
        self.position = position
        self.value = value
        self.domain1 = set(range(10))
        self.domain2 = set(range(10))

    def reduce_domain(self, variable2):
        if variable2 in self.variable1.domain:
            self.domain1.intersection_update(self.variable1.domain)
            self.domain2.intersection_update(self.variable1.domain)

    def reduce_domain1_intersection(self):
        self.domain1.intersection_update(self.variable1.domain)

    def reduce_domain2_intersection(self):
        self.domain2.intersection_update(self.variable1.domain)
        
class Cryptarithmetic:
    def __init__(self, equation):
        self.equation = equation
        self.variables = {}
        self.domains = {}
        self.constraints = {}
        self.assignments = {}
        self.propagate_rules = []

        # Define the variables
        for letter in equation:
            if letter.isalpha() and letter not in self.assignments:
                self.variables[letter] = Variable(letter)
                self.domains[letter] = set(range(10))

        # Define the constraints
        for i, (a, b, c) in enumerate(zip(equation[:-1], equation[1:], equation[2:])):
            if a.isalpha() and b.isalpha() and c.isdigit():
                self.constraints[(a, i)] = Constraint(a, i, int(c) - int(b))

        # Define the propagation rules
        self.propagate_rules.append(self.rule_unique_value)
        self.propagate_rules.append(self.rule_sum_domain)
        self.propagate_rules.append(self.rule_difference_domain)

    def rule_unique_value(self):
        for variable in self.variables.values():
            if len(variable.domain) == 1:
                self.assignments[variable.name] = variable.domain.pop()
                for constraint in self.constraints.values():
                    constraint.reduce_domain(variable.name)

    def rule_sum_domain(self):
        for constraint in self.constraints.values():
            if len(constraint.domain1.intersection(constraint.domain2)) == 0:
                return
            if constraint.value <= sum(constraint.domain1.intersection(constraint.domain2)):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def rule_difference_domain(self):
        for constraint in self.constraints.values():
            if max(constraint.domain1) - min(constraint.domain2) > constraint.value - sum(constraint.domain2):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def propagate(self):
        for rule in self.propagate_rules:
            rule()

    def consistent(self):
        for variable in self.variables.values():
            if len(variable.domain) == 0:
                return False
        return True

    def solved(self):
        return len(self.assignments) == len(self.variables)
    
    def search(self, depth_limit=1):
        if not self.consistent():
            return False
        if self.solved():
            self.solutions.append(self.assignments.copy())
            return True

        variable = min(self.variables.values(), key=lambda x: len(x.domain))
        if len(variable.domain) == 0:
            return False

        for value in variable.domain:
            variable.assign(value)
            self.propagate()
            if self.consistent():
                if self.search(depth_limit + 1):
                    return True
            self.backtrack()

        variable.unassign()
        return False

    def backtrack(self):
        for variable in self.variables.values():
            if variable.assigned():
                variable.unassign()
                self.propagate()
                if not self.consistent():
                    return False
        return True

In [49]:
eq = "SEND + MORE = MONEY"
crypt = Cryptarithmetic(eq)
crypt.search()

for variable, value in crypt.assignments.items():
    print(f"{variable} = {value}")

S = 9


In [44]:
class Variable:
    def __init__(self, name):
        self.name = name
        self.domain = set(range(10))
        self._assigned = False

    def assigned(self):
        return self._assigned

    def unassign(self):
        self._assigned = False

    def assign(self, value):
        self.domain = {value}
        self._assigned = True

class Constraint:
    def __init__(self, variable1, position, value):
        self.variable1 = variable1
        self.position = position
        self.value = value
        self.domain1 = set(range(10))
        self.domain2 = set(range(10))

    def reduce_domain(self, variable2):
        if variable2 in self.variable1.domain:
            self.domain1.intersection_update(self.variable1.domain)
            self.domain2.intersection_update(self.variable1.domain)

    def reduce_domain1_intersection(self):
        self.domain1.intersection_update(self.variable1.domain)

    def reduce_domain2_intersection(self):
        self.domain2.intersection_update(self.variable1.domain)

class Cryptarithmetic:
    def __init__(self, equation):
        self.equation = equation
        self.variables = {}
        self.domains = {}
        self.constraints = {}
        self.assignments = {}
        self.solutions = []  # List to store all solutions
        self.propagate_rules = []

        # Define the variables
        for letter in equation:
            if letter.isalpha() and letter not in self.assignments:
                self.variables[letter] = Variable(letter)
                self.domains[letter] = set(range(10))

        # Define the constraints (use dictionary comprehension for efficiency)
        self.constraints = {
            (a, i): Constraint(a, i, int(c) - int(b))
            for i, (a, b, c) in enumerate(zip(equation[:-1], equation[1:], equation[2:]))
            if a.isalpha() and b.isalpha() and c.isdigit()
        }

        # Define the propagation rules
        self.propagate_rules.append(self.rule_unique_value)
        self.propagate_rules.append(self.rule_sum_domain)
        self.propagate_rules.append(self.rule_difference_domain)

    def rule_unique_value(self):
        for variable in self.variables.values():
            if len(variable.domain) == 1:
                self.assignments[variable.name] = variable.domain.pop()
                for constraint in self.constraints.values():
                    constraint.reduce_domain(variable.name)

    def rule_sum_domain(self):
        for constraint in self.constraints.values():
            if len(constraint.domain1.intersection(constraint.domain2)) == 0:
                return
            if constraint.value <= sum(min(constraint.domain1), min(constraint.domain2)):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def rule_difference_domain(self):
        for constraint in self.constraints.values():
            if max(constraint.domain1) - min(constraint.domain2) > constraint.value - sum(constraint.domain2):
                constraint.reduce_domain1_intersection()
                constraint.reduce_domain2_intersection()

    def propagate(self):
        for rule in self.propagate_rules:
            rule()

    def consistent(self):
        for variable in self.variables.values():
            if len(variable.domain) == 0:
                return False
        return True

    def solved(self):
        return len(self.assignments) == len(self.variables)

    def search(self, depth_limit=1):
        if not self.consistent():
            return False
        if self.solved():
            self.solutions.append(self.assignments.copy())
            return True

        variable = min(self.variables.values(), key=lambda x: len(x.domain))
        if len(variable.domain) == 0:
            return False

        for value in variable.domain:
            variable.assign(value)
            self.propagate()
            if self.consistent():
                # Recursive call to explore this assignment further
                if self.search(depth_limit + 1):
                    return True
        # Backtrack: undo the assignment and try other values
            self.backtrack()

        variable.unassign()
        return False

    def backtrack(self):
        for variable in self.variables.values():
            if variable.assigned():
                variable.unassign()
                self.propagate()
                if not self.consistent():
                    return False
        return True


In [ ]:
eq = "SEND + MORE = MONEY"
crypt = Cryptarithmetic(eq)
solutions = crypt.search()

if solutions:
    for solution in solutions:
        for var, value in solution.items():
            print(f"{var} = {value}")
        print("---")
else:
    print("No solutions found!")


No solutions found!


# Parte Emiliano Sandoval

In [2]:
class Variable:
    """A class to represent a variable in the cryptoarithmetic solver with a domain and assigned value."""
    def __init__(self, name):
        self.name = name
        self.domain = set(range(10))  # Each variable can potentially be any digit from 0-9
        self.assigned_value = None

    def assign(self, value):
        """Assign a value to the variable, limiting its domain to this single value."""
        self.domain = {value}
        self.assigned_value = value

    def unassign(self):
        """Reset the variable, making it unassigned and its domain full range again."""
        self.domain = set(range(10))
        self.assigned_value = None

    def is_assigned(self):
        """Check if the variable has an assigned value."""
        return self.assigned_value is not None

In [3]:
class CryptoarithmeticSolver:
    """A solver for cryptoarithmetic problems using constraint satisfaction and backtracking."""
    def __init__(self, equation):
        self.equation = equation.replace(" ", "")
        self.variables = {char: Variable(char) for char in set(self.equation) if char.isalpha()}
        self.operands, self.result = self.parse_equation()
        self.constraints = []  # Not utilized in this simple example but could be for complex constraints

    def parse_equation(self):
        """Parse the input equation into operands and result."""
        left, right = self.equation.split('=')
        operands = left.split('+')
        return operands, right

    def setup_domains(self):
        """Remove zero from the domain of variables that cannot be zero (leading digits)."""
        for operand in self.operands + [self.result]:
            if len(operand) > 1:
                self.variables[operand[0]].domain.discard(0)

    def all_different_constraint(self):
        """Ensure that all assigned variables have unique values."""
        used_digits = set()
        for variable in self.variables.values():
            if variable.is_assigned():
                if variable.assigned_value in used_digits:
                    return False
                used_digits.add(variable.assigned_value)
        return True

    def solve(self):
        """Attempt to solve the puzzle using backtracking and constraint satisfaction."""
        if not self.all_different_constraint():
            return False

        unassigned_vars = [var for var in self.variables.values() if not var.is_assigned()]
        if not unassigned_vars:  # If all variables are assigned, check if the solution is correct
            return self.is_solved()

        variable = min(unassigned_vars, key=lambda x: len(x.domain))
        for value in list(variable.domain):
            variable.assign(value)
            if self.solve():
                return True
            variable.unassign()
        return False

    def is_solved(self):
        """Check if the current assignments satisfy the equation."""
        operand_values = [self.evaluate_operand(operand) for operand in self.operands]
        result_value = self.evaluate_operand(self.result)
        if None in operand_values or result_value is None:
            return False
        return sum(operand_values) == result_value

    def evaluate_operand(self, operand):
        """Convert the operand from letters to its numeric value if all characters are assigned."""
        if all(self.variables[char].is_assigned() for char in operand):
            return int(''.join(str(self.variables[char].assigned_value) for char in operand))
        else:
            return None
        
    def evaluate_result(self):
        """Evaluate and print the result of the cryptoarithmetic equation."""
        operand_values = [self.evaluate_operand(operand) for operand in self.operands]
        result_value = self.evaluate_operand(self.result)
        if None not in operand_values and result_value is not None:
            computed_sum = sum(operand_values)
            print(f"Computed Sum of Operands: {computed_sum}")
            print(f"Value of Result: {result_value}")
            print("Equation holds: " + ("True" if computed_sum == result_value else "False"))
        else:
            print("Equation cannot be evaluated: Incomplete assignments.")


In [4]:
equation = "SEND + MORE = MONEY"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

R = 8
O = 0
E = 5
Y = 2
N = 6
M = 1
D = 7
S = 9


In [22]:
solver.evaluate_result()

Computed Sum of Operands: 3185
Value of Result: 3185
Equation holds: True


In [5]:
# Pruebas 

In [6]:
equation = "SEND + MORE = MONEY"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

R = 8
O = 0
E = 5
Y = 2
N = 6
M = 1
D = 7
S = 9


In [7]:
equation = "TWO + TWO = FOUR"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

R = 8
T = 2
O = 4
U = 6
W = 3
F = 0


In [8]:
equation = "CAFE + BEAD = DEAF"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

No solution found.


In [15]:
equation = "CROSS + ROADS = DANGER"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

R = 6
O = 2
E = 4
G = 7
N = 8
A = 5
D = 1
S = 3
C = 9


In [10]:
equation = "Base + ball = games"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

e = 4
m = 7
b = 2
g = 0
a = 3
l = 5
B = 1
s = 9


In [11]:
equation = "EAT + THAT = APPLE"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

T = 9
E = 8
P = 0
A = 1
H = 2
L = 3


In [12]:
equation = "CAR + CAR = TRUCK"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

No solution found.


In [13]:
equation = "DIGIT + DIGIT = NUMBER"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

I = 3
R = 4
T = 2
U = 0
E = 6
G = 9
N = 1
M = 7
D = 5
B = 8


In [14]:
equation = "TEN + HER = NEW"
solver = CryptoarithmeticSolver(equation)
solver.setup_domains()
if solver.solve():
    for char, var in solver.variables.items():
        print(f"{char} = {var.assigned_value}")
else:
    print("No solution found.")

R = 4
T = 1
E = 0
N = 3
W = 7
H = 2


In [22]:
def generate_problems(num_problems):
    """Generate and solve a specified number of cryptoarithmetic problems."""
    solved_count = 0
    for i in range(num_problems):
        puzzle = generate_puzzle()  # Generate a new puzzle
        solver = CryptoarithmeticSolver(puzzle)
        if solver.solve():
            solved_count += 1
    return solved_count

In [23]:
import random 
def generate_puzzle():
    """Generate a random cryptoarithmetic puzzle."""
    # Define a list of words to choose from
    words = ["SEND", "MORE", "MONEY", "WATER", "JUICE", "SODA", "CANDY", "SNACK", "COOKIE"]
    # Randomly select three words from the list
    selected_words = random.sample(words, 3)
    # Join the words into a puzzle equation
    puzzle = " + ".join(selected_words[:-1]) + " = " + selected_words[-1]
    return puzzle

# Define the number of problems to generate and solve
num_problems = 10
# Generate and solve the problems
solved_count = generate_problems(num_problems)
print(f"Solved {solved_count} out of {num_problems} problems.")

KeyboardInterrupt: 